# 3-MutationsToStructure
In this tutorial we create and join various datasets to analyse missense mutations.

1. Calculate protein-ligand interactions for [Imatinib](https://en.wikipedia.org/wiki/Imatinib) (Gleevec)
2. Join the results with the SIFTS dataset to obtain UniProt mappings
3. Use the UniProt mappings to retrieve pathogenic mutations from [ClinVar](https://www.ncbi.nlm.nih.gov/clinvar/)
4. Map the mutations from genome locations to 3D PDB positions
5. Visualize the locations of the mutations on the 3D structure

In [1]:
from pyspark import SparkContext
from pyspark.sql import Row, SparkSession
from pyspark.sql.functions import concat_ws
from mmtfPyspark.datasets import g2sDataset, pdbjMineDataset, myVariantDataset
from mmtfPyspark.filters import ContainsGroup
from mmtfPyspark.interactions import InteractionFilter, InteractionFingerprinter
from mmtfPyspark.io import mmtfReader
from ipywidgets import interact, IntSlider
import py3Dmol

#### Configure Spark Session

In [2]:
spark = SparkSession.builder.master("local[4]").appName("Solution-1").getOrCreate()
sc = spark.sparkContext

#### Read a  sample of the PDB

In [3]:
path = "../resources/mmtf_full_sample"
pdb = mmtfReader.read_sequence_file(path)

## 1. Calculate protein-ligand interactions for Imatinib (Gleevec)  `STI`

In [4]:
pdb = pdb.filter(ContainsGroup(("STI")))

#### [InteractionFilter](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/mmtfPyspark/interactions/interactionFilter.py) sets criteria for finding interactions

In [5]:
interactionFilter = InteractionFilter()
interactionFilter.set_distance_cutoff(4.5)
interactionFilter.set_min_interactions(1)
interactionFilter.set_query_groups(True, ["STI"])

#### [InteractionFingerprinter](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/mmtfPyspark/interactions/interactionFingerprinter.py) finds ligand-polymer interactions that meet the filter criteria

In [6]:
interactions = InteractionFingerprinter.get_ligand_polymer_interactions(pdb, interactionFilter).cache()

In [7]:
interactions.toPandas().head(5)

,structureChainId,queryLigandId,queryLigandNumber,queryLigandChainId,targetChainId,groupNumbers,sequenceIndices,sequence,interactingChains
0,2OIQ.A,STI,1001,A,A,"[273, 281, 293, 294, 295, 310, 313, 314, 317, ...","[25, 33, 45, 46, 47, 62, 65, 66, 69, 74, 75, 8...",GHMQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVA...,1
1,5MQT.A,STI,302,A,A,"[30, 53, 55, 82, 85, 86, 89, 90, 96, 97, 128, ...","[58, 81, 83, 110, 113, 114, 117, 118, 124, 125...",MSYYHHHHHHLESTSLYKKAGLENLYFQGMATPPKRSSPSFSASSE...,1
2,5MQT.C,STI,302,C,C,"[30, 53, 55, 82, 85, 86, 89, 90, 96, 97, 128, ...","[58, 81, 83, 110, 113, 114, 117, 118, 124, 125...",MSYYHHHHHHLESTSLYKKAGLENLYFQGMATPPKRSSPSFSASSE...,1
3,1T46.A,STI,3,A,A,"[595, 603, 621, 622, 623, 640, 643, 644, 647, ...","[30, 38, 56, 57, 58, 75, 78, 79, 82, 89, 103, ...",GNNYVYIDPTQLPYDHKWEFPRNRLSFGKTLGAGAFGKVVEATAYG...,1
4,3HEC.A,STI,1,A,A,"[30, 38, 51, 52, 53, 71, 74, 75, 84, 104, 106,...","[25, 33, 46, 47, 48, 66, 69, 70, 79, 99, 101, ...",RPTFYRQELNKTIWEVPERYQNLSPVGSGAYGSVCAAFDTKTGLRV...,1


## 2. Join the results with the SIFTS dataset to obtain UniProt mappings

### Get up-to-date PDB chain to UniProt mapping from the SIFTS projects

In [8]:
uniprotQuery = "SELECT * FROM sifts.pdb_chain_uniprot"
uniprot = pdbjMineDataset.get_dataset(uniprotQuery)

In [9]:
uniprot.toPandas().head(5)

,pdbid,chain,sp_primary,res_beg,res_end,pdb_beg,pdb_end,sp_beg,sp_end,structureChainId
0,101M,A,P02185,1,154,0,153,1,154,101M.A
1,102L,A,P00720,1,40,1,40,1,40,102L.A
2,102L,A,P00720,42,165,41,None,41,164,102L.A
3,102M,A,P02185,1,154,0,153,1,154,102M.A
4,103L,A,P00720,1,40,1,None,1,40,103L.A


### Map interacting chains to UniProt data

In [10]:
intersection = uniprot.join(interactions, uniprot.structureChainId == interactions.structureChainId).cache()

In [11]:
intersection.toPandas()

,pdbid,chain,sp_primary,res_beg,res_end,pdb_beg,pdb_end,sp_beg,sp_end,structureChainId,structureChainId,queryLigandId,queryLigandNumber,queryLigandChainId,targetChainId,groupNumbers,sequenceIndices,sequence,interactingChains
0,1IEP,A,P00520,7,293,229,None,229,515,1IEP.A,1IEP.A,STI,201,A,A,"[248, 253, 256, 269, 270, 271, 286, 289, 290, ...","[25, 30, 33, 46, 47, 48, 63, 66, 67, 70, 76, 9...",GAMDPSSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTV...,1
1,1IEP,B,P00520,7,293,229,None,229,515,1IEP.B,1IEP.B,STI,202,B,B,"[248, 253, 256, 269, 270, 271, 286, 289, 290, ...","[25, 30, 33, 46, 47, 48, 63, 66, 67, 70, 76, 9...",GAMDPSSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTV...,1
2,1T46,A,P10721,1,313,565,None,565,935,1T46.A,1T46.A,STI,3,A,A,"[595, 603, 621, 622, 623, 640, 643, 644, 647, ...","[30, 38, 56, 57, 58, 75, 78, 79, 82, 89, 103, ...",GNNYVYIDPTQLPYDHKWEFPRNRLSFGKTLGAGAFGKVVEATAYG...,1
3,1XBB,A,P43405,4,283,None,635,356,635,1XBB.A,1XBB.A,STI,1,A,A,"[377, 378, 379, 385, 400, 433, 448, 449, 450, ...","[24, 25, 26, 32, 47, 80, 95, 96, 97, 98, 99, 1...",MALEEIRPKEVYLDRKLLTLEDKELGSGNFGTVKKGYYQMKKVVKT...,1
4,2HYY,A,P00519,1,273,None,None,228,500,2HYY.A,2HYY.A,STI,600,A,A,"[248, 253, 256, 269, 270, 271, 286, 289, 290, ...","[20, 25, 28, 41, 42, 43, 58, 61, 62, 65, 71, 8...",VSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTL...,1
5,2HYY,B,P00519,1,273,None,None,228,500,2HYY.B,2HYY.B,STI,600,B,B,"[248, 253, 256, 269, 270, 271, 286, 289, 290, ...","[20, 25, 28, 41, 42, 43, 58, 61, 62, 65, 71, 8...",VSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTL...,1
6,2HYY,C,P00519,1,273,None,None,228,500,2HYY.C,2HYY.C,STI,600,C,C,"[248, 253, 256, 269, 270, 271, 286, 289, 290, ...","[20, 25, 28, 41, 42, 43, 58, 61, 62, 65, 71, 8...",VSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTL...,1
7,2HYY,D,P00519,1,273,None,None,228,500,2HYY.D,2HYY.D,STI,600,D,D,"[248, 253, 256, 269, 270, 271, 286, 289, 290, ...","[20, 25, 28, 41, 42, 43, 58, 61, 62, 65, 71, 8...",VSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTVAVKTL...,1
8,2OIQ,A,P00523,4,286,None,533,251,533,2OIQ.A,2OIQ.A,STI,1001,A,A,"[273, 281, 293, 294, 295, 310, 313, 314, 317, ...","[25, 33, 45, 46, 47, 62, 65, 66, 69, 74, 75, 8...",GHMQTQGLAKDAWEIPRESLRLEVKLGQGCFGEVWMGTWNGTTRVA...,1
9,2PL0,A,P06239,5,289,None,None,225,509,2PL0.A,2PL0.A,STI,200,A,A,"[259, 271, 272, 273, 288, 291, 292, 295, 300, ...","[38, 50, 51, 52, 67, 70, 71, 74, 79, 80, 93, 9...",GSHMQTQKPQKPWWEDEWEVPRETLKLVERLGAGQFGEVWMGYYNG...,1


### Extract UniProt Ids as a list

In [12]:
uniprot_ids = intersection.select("sp_primary").distinct().rdd.flatMap(lambda x: x).collect()
uniprot_ids

['Q16539',
 'P07333',
 'P43405',
 'P00519',
 'P00520',
 'P06239',
 'Q08345',
 'P42684',
 'P00523',
 'P27707',
 'P16083',
 'P10721']

## 3. Use the UniProt mappings to retrieve pathogenic mutations from ClinVar
Here we use [myVariantDataset](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/mmtfPyspark/datasets/myVariantDataset.py) to retrieve variant information using [MyVariant.info](http://myvariant.info) web services ([Query syntax](http://myvariant.info/docs))

In [13]:
query = "clinvar.rcv.clinical_significance:pathogenic"
variants = myVariantDataset.get_variations(uniprot_ids, query).cache()
variants.toPandas()

,variationId,uniprotId
0,chr5:g.149435602C>G,P07333
1,chr5:g.149434851A>G,P07333
2,chr5:g.149440497C>T,P07333
3,chr5:g.149434892A>T,P07333
4,chr5:g.149435693A>G,P07333
5,chr5:g.149440505A>C,P07333
6,chr5:g.149436872A>G,P07333
7,chr5:g.149435675G>A,P07333
8,chr5:g.149435900A>T,P07333
9,chr5:g.149434822G>T,P07333


### Extract Variant Ids as a list

In [14]:
variant_ids = variants.select("variationId").distinct().rdd.flatMap(lambda x: x).collect()
variant_ids

['chr5:g.149441146C>T',
 'chr4:g.55593608G>C',
 'chr9:g.133747540T>G',
 'chr9:g.133738306G>A',
 'chr5:g.149435597A>G',
 'chr9:g.133748290C>A',
 'chr4:g.55593606A>C',
 'chr5:g.149440436C>T',
 'chr9:g.133748391T>C',
 'chr5:g.149440505A>C',
 'chr9:g.133748414T>A',
 'chr1:g.32745329T>C',
 'chr9:g.133738364A>T',
 'chr4:g.55599332G>C',
 'chr4:g.55561764G>A',
 'chr5:g.149453043C>A',
 'chr5:g.149436872A>G',
 'chr9:g.133748270T>C',
 'chr4:g.55592186T>C',
 'chr5:g.149436875C>T',
 'chr4:g.55599320G>T',
 'chr5:g.149435882G>T',
 'chr4:g.55599340T>G',
 'chr5:g.149441328A>C',
 'chr5:g.149435634C>A',
 'chr4:g.55592144G>A',
 'chr4:g.55593605G>C',
 'chr4:g.55594261A>G',
 'chr4:g.55599320G>C',
 'chr4:g.55593608G>T',
 'chr5:g.149435602C>G',
 'chr4:g.55593610T>A',
 'chr4:g.55599334T>A',
 'chr4:g.55593603T>G',
 'chr5:g.149434888A>G',
 'chr5:g.149434851A>G',
 'chr9:g.133738356G>T',
 'chr5:g.149435693A>G',
 'chr4:g.55594262T>A',
 'chr9:g.133750356A>G',
 'chr4:g.55594073T>C',
 'chr4:g.55599333A>G',
 'chr9:g.13

## 4. Map the mutations from genome locations to 3D PDB positions
Here we use [g2sDataset](https://github.com/sbl-sdsc/mmtf-pyspark/blob/master/mmtfPyspark/datasets/g2sDataset.py) to retrieve genome to PDB mapping information using the [G2S]( https://g2s.genomenexus.org) (Genome to Structure) web services.

In [15]:
positions = g2sDataset.get_position_dataset(variant_ids).cache()
positions.toPandas()

,structureId,chainId,pdbPosition,pdbAminoAcid,refGenome,variationId
0,2HZN,A,283,F,hgvs-grch37,chr9:g.133747540T>G
1,2OGV,A,653,C,hgvs-grch37,chr5:g.149440436C>T
2,4UWY,A,709,D,hgvs-grch37,chr4:g.55599332G>C
3,3KRL,A,766,M,hgvs-grch37,chr5:g.149436872A>G
4,4J98,B,470,E,hgvs-grch37,chr5:g.149441328A>C
5,4UWC,A,764,F,hgvs-grch37,chr5:g.149434851A>G
6,4U0I,A,820,D,hgvs-grch37,chr4:g.55599333A>G
7,3KXX,B,465,L,hgvs-grch37,chr4:g.55593661T>C
8,4RWK,A,465,L,hgvs-grch37,chr4:g.55593661T>C
9,4RWJ,A,531,E,hgvs-grch37,chr4:g.55594223A>C


#### Concatenate structureId and chainId columns to structureChainId (required for join operation below)
See documentation for [concat_ws](http://spark.apache.org/docs/2.1.0/api/python/pyspark.sql.html#pyspark.sql.functions.concat_ws)

In [16]:
positions = positions.withColumn("structureChainId", concat_ws(".", positions.structureId,positions.chainId))

In [17]:
positions.toPandas().head(5)

,structureId,chainId,pdbPosition,pdbAminoAcid,refGenome,variationId,structureChainId
0,2HZN,A,283,F,hgvs-grch37,chr9:g.133747540T>G,2HZN.A
1,2OGV,A,653,C,hgvs-grch37,chr5:g.149440436C>T,2OGV.A
2,4UWY,A,709,D,hgvs-grch37,chr4:g.55599332G>C,4UWY.A
3,3KRL,A,766,M,hgvs-grch37,chr5:g.149436872A>G,3KRL.A
4,4J98,B,470,E,hgvs-grch37,chr5:g.149441328A>C,4J98.B


### Join mutation data with the structures in the sample PDB set that interact with Imatinib

In [18]:
positions = positions.join(interactions, positions.structureChainId == interactions.structureChainId)
positions.toPandas()

,structureId,chainId,pdbPosition,pdbAminoAcid,refGenome,variationId,structureChainId,structureChainId,queryLigandId,queryLigandNumber,queryLigandChainId,targetChainId,groupNumbers,sequenceIndices,sequence,interactingChains
0,4R7I,A,849,F,hgvs-grch37,chr5:g.149435597A>G,4R7I.A,4R7I.A,STI,1001,A,A,"[588, 596, 614, 615, 616, 633, 636, 637, 640, ...","[57, 65, 83, 84, 85, 102, 105, 106, 109, 116, ...",MKKGHHHHHHGQKPKYQVRWKIIESYEGNSYTFIDPTQLPYNEKWE...,1
1,4R7I,A,806,D,hgvs-grch37,chr4:g.55599334T>A,4R7I.A,4R7I.A,STI,1001,A,A,"[588, 596, 614, 615, 616, 633, 636, 637, 640, ...","[57, 65, 83, 84, 85, 102, 105, 106, 109, 116, ...",MKKGHHHHHHGQKPKYQVRWKIIESYEGNSYTFIDPTQLPYNEKWE...,1
2,4R7I,A,901,P,hgvs-grch37,chr5:g.149433947G>A,4R7I.A,4R7I.A,STI,1001,A,A,"[588, 596, 614, 615, 616, 633, 636, 637, 640, ...","[57, 65, 83, 84, 85, 102, 105, 106, 109, 116, ...",MKKGHHHHHHGQKPKYQVRWKIIESYEGNSYTFIDPTQLPYNEKWE...,1
3,1T46,A,566,N,hgvs-grch37,chr4:g.55593630A>G,1T46.A,1T46.A,STI,3,A,A,"[595, 603, 621, 622, 623, 640, 643, 644, 647, ...","[30, 38, 56, 57, 58, 75, 78, 79, 82, 89, 103, ...",GNNYVYIDPTQLPYDHKWEFPRNRLSFGKTLGAGAFGKVVEATAYG...,1
4,1IEP,B,236,E,hgvs-grch37,chr9:g.133738307A>T,1IEP.B,1IEP.B,STI,202,B,B,"[248, 253, 256, 269, 270, 271, 286, 289, 290, ...","[25, 30, 33, 46, 47, 48, 63, 66, 67, 70, 76, 9...",GAMDPSSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTV...,1
5,1IEP,B,283,F,hgvs-grch37,chr9:g.133747540T>G,1IEP.B,1IEP.B,STI,202,B,B,"[248, 253, 256, 269, 270, 271, 286, 289, 290, ...","[25, 30, 33, 46, 47, 48, 63, 66, 67, 70, 76, 9...",GAMDPSSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTV...,1
6,1IEP,A,359,F,hgvs-grch37,chr9:g.133748414T>C,1IEP.A,1IEP.A,STI,201,A,A,"[248, 253, 256, 269, 270, 271, 286, 289, 290, ...","[25, 30, 33, 46, 47, 48, 63, 66, 67, 70, 76, 9...",GAMDPSSPNYDKWEMERTDITMKHKLGGGQYGEVYEGVWKKYSLTV...,1
7,1T46,A,620,V,hgvs-grch37,chr4:g.55594073T>C,1T46.A,1T46.A,STI,3,A,A,"[595, 603, 621, 622, 623, 640, 643, 644, 647, ...","[30, 38, 56, 57, 58, 75, 78, 79, 82, 89, 103, ...",GNNYVYIDPTQLPYDHKWEFPRNRLSFGKTLGAGAFGKVVEATAYG...,1
8,1T46,A,576,L,hgvs-grch37,chr4:g.55593661T>C,1T46.A,1T46.A,STI,3,A,A,"[595, 603, 621, 622, 623, 640, 643, 644, 647, ...","[30, 38, 56, 57, 58, 75, 78, 79, 82, 89, 103, ...",GNNYVYIDPTQLPYDHKWEFPRNRLSFGKTLGAGAFGKVVEATAYG...,1
9,4R7I,A,847,E,hgvs-grch37,chr5:g.149435602C>G,4R7I.A,4R7I.A,STI,1001,A,A,"[588, 596, 614, 615, 616, 633, 636, 637, 640, ...","[57, 65, 83, 84, 85, 102, 105, 106, 109, 116, ...",MKKGHHHHHHGQKPKYQVRWKIIESYEGNSYTFIDPTQLPYNEKWE...,1


## 5. Visualize the locations of the mutations on the 3D structure

#### Create lists of ids

In [19]:
pdb_ids = positions.select("structureId").rdd.flatMap(lambda x: x).collect()
chain_ids = positions.select("chainId").rdd.flatMap(lambda x: x).collect()
group_numbers = positions.select("pdbPosition").rdd.flatMap(lambda x: x).collect()
var_ids = positions.select("variationId").rdd.flatMap(lambda x: x).collect()

#### Custom method to visualize the site of a mutation

In [20]:
def view_mutation_site(pdb_ids, chain_ids, groups, var_ids, ligand_id, distance=4.5):
    
    def view3d(i=0):
        
        print(f"PDB: {pdb_ids[i]}, chain: {chain_ids[i]}, group: {groups[i]}, variation: {var_ids[i]}")

        mutation = {'resi': groups[i], 'chain': chain_ids[i]}
        neighbors = {'resi': groups[i], 'chain': chain_ids[i], 'byres': 'true', 'expand': distance}

        viewer = py3Dmol.view(query='pdb:' + pdb_ids[i]) 
        viewer.setStyle(neighbors, {'stick': {'colorscheme': 'orangeCarbon', 'radius': 0.1}});
        viewer.setStyle(mutation, {'stick': {'colorscheme': 'redCarbon'}})
        viewer.addResLabels(mutation)
        viewer.setStyle({'resn': ligand_id}, {'sphere': {'colorscheme': 'greenCarbon'}})
        viewer.zoomTo(neighbors)

        return viewer.show()

    s_widget = IntSlider(min=0, max=len(pdb_ids)-1, description='Structure', continuous_update=False)
    return interact(view3d, i=s_widget)

#### The mutation site is rendered as a red stick. Residues within 8 A of the mutation site are rendered as thin orange stickes. Imatinib (Gleevec) is rendered in as spheres.

In [21]:
%%javascript 
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

<IPython.core.display.Javascript object>

In [22]:
view_mutation_site(pdb_ids, chain_ids, group_numbers, var_ids, 'STI', 8.0);

interactive(children=(IntSlider(value=0, continuous_update=False, description='Structure', max=151), Output())…

In [23]:
spark.stop()